In [ ]:
# Configure sys.path so src/ and ArtWork/ modules are importable
import sys, importlib
from pathlib import Path
repo_root = Path.cwd().resolve()
while repo_root != repo_root.parent and not ((repo_root / 'src').exists() and (repo_root / 'ArtWork').exists()):
    repo_root = repo_root.parent
if not (repo_root / 'src').exists():
    raise RuntimeError('Unable to locate XMODE repository root')
artwork_dir = repo_root / 'ArtWork'
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
if str(artwork_dir) not in sys.path:
    sys.path.insert(1, str(artwork_dir))
sys.modules['tools'] = importlib.import_module('ArtWork.tools')
sys.modules['tools.backend'] = importlib.import_module('ArtWork.tools.backend')


In [ ]:
# Install notebook dependencies (Claude, LangChain, plotting, etc.)
%%capture --no-stderr
%pip install -U --quiet langchain-openai langchain-anthropic langchain langchainhub langsmith langchain-community langgraph numexpr sqlalchemy matplotlib torch==2.0.0 torchvision==0.15.1 transformers sentence-transformers[train]==3.0.1

In [ ]:
# Generate the isolated test dataset (image + SQLite DB + questions)
import subprocess, sys
subprocess.run([sys.executable, 'bhy_test/setup_environment.py'], check=True)


In [ ]:
# Load Settings and point Artwork config to the bhy_test assets
import os, getpass
from src.settings import get_settings
settings = get_settings()
TEST_BASE = settings.base_dir / 'bhy_test'
ARTWORK_CFG = settings.artwork
ARTWORK_CFG.db_path = TEST_BASE / 'nature.db'
ARTWORK_CFG.questions_file = TEST_BASE / 'questions_nature.json'
ARTWORK_CFG.log_dir = TEST_BASE / 'logs'
ARTWORK_CFG.log_dir.mkdir(parents=True, exist_ok=True)
ARTWORK_CFG.thread_id = 'bhy-test'

def _set_if_missing(var):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f'Provide {var}: ')

_set_if_missing('OPENAI_API_KEY')
if settings.models.provider.lower() == 'anthropic':
    _set_if_missing('ANTHROPIC_API_KEY')
print('Using DB:', ARTWORK_CFG.db_path)


In [ ]:
# Build the LangGraph planner/executor with the isolated DB
from ArtWork.src.build_graph import graph_construction
provider = settings.models.provider.lower()
model = settings.models.default_chat_model if provider == 'openai' else settings.models.anthropic_model
temperature = settings.models.default_temperature
chain = graph_construction(
    model=model,
    temperature=temperature,
    db_path=str(ARTWORK_CFG.db_path),
    log_path=str(ARTWORK_CFG.log_dir),
    vqa_provider=settings.artwork.vqa_provider,
    settings=settings,
)
print('Graph ready with provider', settings.models.provider, 'and VQA', settings.artwork.vqa_provider)


In [ ]:
# Run a sample question against the test DB and stream intermediate steps
from langchain_core.messages import HumanMessage
from src.utils import _get_db_schema
schema = _get_db_schema(str(ARTWORK_CFG.db_path))
question = 'Describe the Serene Valley painting in the test dataset.'
config = {'configurable': {'thread_id': 'bhy-test-demo'}}
print('Question:', question)
for step in chain.stream(question, config, stream_mode='values'):
    for message in step:
        print(message)
    print('---')
